# lightGBM - Greedy Bundling

- Modified from: https://github.com/meanxai/machine_learning/blob/main/12.LGBM/3.greedy_bundling.py
- A detailed description of this code can be found in https://youtu.be/Y-IvfsjmqOQ

In [1]:
import numpy as np

x = np.array(
    [
        [1, 1, 0, 0, 1],
        [0, 0, 1, 1, 1],
        [1, 2, 0, 0, 2],
        [0, 0, 2, 3, 1],
        [2, 1, 0, 0, 3],
        [3, 3, 0, 0, 1],
        [0, 0, 3, 0, 2],
        [1, 2, 3, 4, 3],
        [1, 0, 1, 0, 0],
        [2, 3, 0, 0, 2],
    ]
)

In [2]:
# Create a conflict count matrix
n_row = x.shape[0]
n_col = x.shape[1]
conflictCnt = np.zeros((n_col, n_col))

for i in range(n_col):
    for j in range(i + 1, n_col):
        # Count the number of conflicts.
        conflictCnt[i, j] = len(np.where(x[:, i] * x[:, j] > 0)[0])
print(conflictCnt)

[[0. 6. 2. 1. 6.]
 [0. 0. 1. 1. 6.]
 [0. 0. 0. 3. 4.]
 [0. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0.]]


In [3]:
# iu = (array([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]),
#       array([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]))
iu = np.triu_indices(n_col, 1)
il = (iu[1], iu[0])
# il = (array([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]),
#       array([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]))
# Copy upper triangle to lower triangle
conflictCnt[il] = conflictCnt[iu]
print(conflictCnt)

[[0. 6. 2. 1. 6.]
 [6. 0. 1. 1. 6.]
 [2. 1. 0. 3. 4.]
 [1. 1. 3. 0. 3.]
 [6. 6. 4. 3. 0.]]


In [4]:
degree = conflictCnt.sum(axis=0)
searchOrder = np.argsort(degree)[::-1]  # descending order
print(degree)
print(searchOrder)

[15. 14. 10.  8. 19.]
[4 0 1 2 3]


In [5]:
# ----------------------------
# Algorithm 3: Greedy Bundling
# ----------------------------
K = 1  # max conflict count
bundles = []
bundlesConflict = []
searchOrder = [int(x) for x in searchOrder]  # just make print better later
for i in searchOrder:  # i = [4, 0, 1, 2, 3]
    needNew = True
    for j in range(len(bundles)):
        cnt = int(conflictCnt[bundles[j][-1], i])
        # Only edges less than or equal to K are considered.
        if cnt + bundlesConflict[j] <= K:
            # Add the feature number i to the j-th bundle.
            bundles[j].append(i)

            # Update the number of conflicts of features in the
            # j-th bundle.
            bundlesConflict[j] += cnt
            needNew = False
            break

    if needNew:
        bundles.append([i])
        bundlesConflict.append(0.0)
    print(f"i={i}, j={j}")
    print(f"\nbundles\n{bundles}")
    print(f"\nbundlesConflict\n{bundlesConflict}")
    print("-----------------------")

print(f"\nconflictCnt:\n{conflictCnt}")
print(f"\nsearchOrder:\n{searchOrder}")

print(f"\nbundles\n{bundles}")
print(f"bundlesConflict: {bundlesConflict}")

i=4, j=4

bundles
[[4]]

bundlesConflict
[0.0]
-----------------------
i=0, j=0

bundles
[[4], [0]]

bundlesConflict
[0.0, 0.0]
-----------------------
i=1, j=1

bundles
[[4], [0], [1]]

bundlesConflict
[0.0, 0.0, 0.0]
-----------------------
i=2, j=2

bundles
[[4], [0], [1, 2]]

bundlesConflict
[0.0, 0.0, 1.0]
-----------------------
i=3, j=1

bundles
[[4], [0, 3], [1, 2]]

bundlesConflict
[0.0, 1.0, 1.0]
-----------------------

conflictCnt:
[[0. 6. 2. 1. 6.]
 [6. 0. 1. 1. 6.]
 [2. 1. 0. 3. 4.]
 [1. 1. 3. 0. 3.]
 [6. 6. 4. 3. 0.]]

searchOrder:
[4, 0, 1, 2, 3]

bundles
[[4], [0, 3], [1, 2]]
bundlesConflict: [0.0, 1.0, 1.0]
